In [3]:
# Import necessary modules
from typing import List
from pydantic import Field

# Import SpoonAI components
from spoon_ai.agents.toolcall import ToolCallAgent # Base class for agents using tools
from spoon_ai.chat import ChatBot
from spoon_ai.tools import ToolManager, Terminate # General tool management and termination tool
from spoon_ai.tools.token_execute.token_transfer import TokenTransfer # The specific tool for this agent


/home/zhangzhihao/miniconda3/envs/backend/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
# --- Define Prompts ---

# System prompt defining the agent's role and capabilities
AGENT_SYSTEM_PROMPT = """You are a specialized AI agent designed to assist with executing token transfers on the blockchain.
Your primary function is to understand user requests for transferring tokens and utilize the 'token_transfer' tool to fulfill these requests.
Always confirm the details (token address, amount, recipient address) before proceeding.
You must use the available tools to answer the query. Stick to the following process:
1. Thought: Briefly reason about the user query and the plan to execute the tool.
2. Action: Determine the action to take using the tools provided. The action must be represented as a JSON object with 'action' and 'action_input' keys.
3. Observation: This will be the result of the action.
4. Final Answer: This is the final response to the user after the process is complete.
Repeat the Thought-Action-Observation cycle until you can provide the Final Answer.
Use the 'terminate' tool when the task is fully completed."""

# Prompt to guide the next step in a multi-step process (if the base class uses it)
# Note: The {agent_scratchpad} placeholder is common in ReAct-style prompts
AGENT_NEXT_STEP_PROMPT = """Based on the previous actions and observations, determine the next step.
If the token transfer is complete and confirmed, use the 'terminate' tool.
Otherwise, formulate the next Thought and Action.
{agent_scratchpad}"""


In [5]:
# --- Define the Agent Class ---

class TokenTransferAgent(ToolCallAgent):
    """
    An AI agent specifically designed to handle token transfer requests 
    by utilizing the TokenTransfer tool. It follows a ReAct-like pattern 
    (Thought-Action-Observation) guided by system prompts.
    """
    
    name: str = "token_transfer_agent"
    description: str = "An agent capable of executing token transfers using the TokenTransfer tool."
    
    # --- Core Agent Configuration ---
    system_prompt: str = AGENT_SYSTEM_PROMPT
    next_step_prompt: str = AGENT_NEXT_STEP_PROMPT # Used if the base class implements ReAct logic
    max_steps: int = 5 # Limit the number of interaction steps
    tool_choice: str = "auto" # Let the LLM decide when to use the tool
    
    # --- LLM and Tools Setup ---
    # Configure the LLM. You might want to customize the ChatBot instance further.
    llm: ChatBot = Field(default_factory=lambda: ChatBot()) 

    # Available tools for this agent: TokenTransfer and Terminate
    available_tools: ToolManager = Field(
        default_factory=lambda: ToolManager([
            TokenTransfer(), 
            Terminate()
        ])
    )
    
    # Specify tools that have special handling or significance (like termination)
    special_tools: List[str] = Field(default=["terminate"])

    # --- Agent Logic (Leveraging Base Class) ---
    # This agent primarily relies on the logic inherited from ToolCallAgent 
    # for the ReAct loop (Thought-Action-Observation) and tool handling.
    # You can override methods from ToolCallAgent here if specific customization 
    # beyond prompt engineering and tool selection is needed.

    # Example of a method you *could* override (but may not need to):
    # def _parse_output(self, output: str) -> dict:
    #     # Custom logic to parse the LLM's output if the default parsing isn't suitable
    #     pass

In [10]:
! export RPC_URL=YOUR_RPC_URL

In [11]:
! export PRIVATE_KEY=YOUR_PRIVATE_KEY

In [12]:
# --- Example Usage (Optional - for testing) ---
if __name__ == '__main__':
    # This block allows testing the agent directly if the file is run as a script.
    # Remember to set necessary environment variables:
    # - LLM API Key (e.g., ANTHROPIC_API_KEY or OPENAI_API_KEY)
    # - Blockchain details (RPC_URL, SCAN_URL, CHAIN_ID)
    # - Wallet details (PRIVATE_KEY) for the TokenTransfer tool to sign transactions.
    
    import asyncio
    import nest_asyncio
    import os

    # Apply nest_asyncio if running in a compatible environment (like Jupyter)
    nest_asyncio.apply()

    async def run_agent_test():
        print("--- Starting Token Transfer Agent Test ---")
        
        # Check for essential environment variables
        required_vars = ['RPC_URL', 'PRIVATE_KEY'] 
        # Add 'ANTHROPIC_API_KEY' or 'OPENAI_API_KEY' based on ChatBot default
        llm_key_set = os.getenv('ANTHROPIC_API_KEY') or os.getenv('OPENAI_API_KEY')
        if not llm_key_set:
             print("WARNING: LLM API Key (ANTHROPIC_API_KEY or OPENAI_API_KEY) not set.")
             # Decide if you want to exit or proceed without LLM functionality
             # return 

        missing_vars = [var for var in required_vars if not os.getenv(var)]
        if missing_vars:
            print(f"ERROR: Missing required environment variables for execution: {', '.join(missing_vars)}")
            print("Agent execution will likely fail.")
            # return # Exit if essential variables for the tool are missing

        # Initialize the agent
        agent = TokenTransferAgent()
        print(f"Agent '{agent.name}' initialized.")
        print(f"Available tools: {[tool.name for tool in agent.available_tools.tools]}")
        
        # Define a sample user request 
        # IMPORTANT: Replace placeholder addresses with actual values for a real test!
        user_request = "Please transfer 0.001 TEST_TOKEN to 0xRecipientPlaceholderAddress. The token address is 0xTokenPlaceholderAddress."
        print(f"\nUser Request: '{user_request}'")
        
        # Execute the agent's run method (assuming it's inherited from ToolCallAgent)
        try:
            if hasattr(agent, 'run'):
                # The exact input format might depend on the ToolCallAgent's run method signature
                result = await agent.run(input=user_request) 
                print("\n--- Agent Execution Finished ---")
                print("Final Result:")
                print(result)
            else:
                print("\nERROR: Agent does not have a 'run' method. Cannot execute.")
        except Exception as e:
            print(f"\n--- Agent Execution Failed ---")
            print(f"An error occurred: {e}")

    # Run the asynchronous test function
    print("Ensure environment variables (API keys, RPC_URL, PRIVATE_KEY, etc.) are set before running.")
    asyncio.run(run_agent_test())
    print("\n(Test run is commented out by default. Uncomment the last line in the script to execute.)") 

Ensure environment variables (API keys, RPC_URL, PRIVATE_KEY, etc.) are set before running.
--- Starting Token Transfer Agent Test ---
ERROR: Missing required environment variables for execution: RPC_URL, PRIVATE_KEY
Agent execution will likely fail.


AttributeError: 'str' object has no attribute 'name'